# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import pickle
import tensorflow as tf
import numpy as np
import string
import random
import pandas as pd


try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [7]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=64, output_dim=64, input_length=35,
                                  embeddings_constraint=tf.keras.constraints.MaxNorm(1)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False, dropout=0.2)),
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dropout(0.20),
        tf.keras.layers.Dense(26, activation='sigmoid')
    ])
    return model

In [8]:
import collections
import re
import pandas as pd
def count_letter_presence(word_list):
    """Count how many words contain each letter in the dictionary.
    
    Args:
        word_list: List of words to process
        
    Returns:
        Counter with letters as keys and count of words containing them as values
    """
    letter_presence = collections.Counter()
    for word in word_list:
        # Use set to get unique letters in word
        unique_letters = set(word)
        # Update counts (each letter counts at most once per word)
        letter_presence.update(unique_letters)
    return letter_presence

def calculate_vowel_ratio(word):
    """Calculate the ratio of vowels in a word.
    
    Args:
        word: Input word to analyze
        
    Returns:
        Float between 0 and 1 representing vowel ratio
    """
    vowels = {'a', 'e', 'i', 'o', 'u'}
    vowel_count = sum(1 for char in word if char in vowels)
    return vowel_count / len(word)

def filter_matching_words(word_dict, pattern_word):
    """Filter words that match the pattern and length of the input word.
    
    Args:
        word_dict: Dictionary of words organized by length
        pattern_word: Word pattern to match (may contain regex special chars)
        
    Returns:
        List of matching words
    """
    word_length = len(pattern_word)
    # Compile regex once for better performance
    pattern = re.compile(pattern_word)
    return [word for word in word_dict.get(word_length, []) 
            if pattern.fullmatch(word)]




In [9]:
vowels = ['a', 'e', 'i', 'o', 'u']
f = open("words_250000_train.txt", "r")
df = []
for x in f:
  df.append(x)
f.close()


for i in range(len(df)):
    df[i] = df[i].replace("\n", "")

l=[]
for words in df:
    l.append(calculate_vowel_ratio(words))
l = pd.Series(l)
l.describe()



max_length = 0
for words in df:
    if(len(words)>max_length):
        max_length = len(words)

n_word_dictionary = {i:[] for i in range(3, 30)}
count = 3
while count<=max_length:
    for words in df:
        if(len(words)>=count):
            for i in range(len(words)-count+1):
                #if words[i:i+count-1] not in n_word_dictionary[count]:
                n_word_dictionary[count].append(words[i:i+count]) 
    count = count+1



In [51]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
        self.alphabet = "abcdefghijklmnopqrstuvwxyz"

        self.tries_remains = 0

        self.model = create_model()
        self.model.build(input_shape=(None, 35))
        self.model.load_weights("bi_lstm.weights.h5")  
        self.model_threshold = 0.65



        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def encode_word(self, masked_word):
        """Encodes the masked word to a 35-length sequence: a-z → 1–26, _ → 27, pad → 0"""
        encoding = {ch: i + 1 for i, ch in enumerate(string.ascii_lowercase)}
        encoding['_'] = 27
        sequence = [encoding.get(ch, 0) for ch in masked_word.lower()]
        sequence = sequence[:35] + [0] * (35 - len(sequence))
        return np.array([sequence])

    def fallback_guess(self, word):
        """Fallback based on global letter frequency."""
        regex_pattern = word.replace("_", ".")
        word_length = len(regex_pattern)
        current_words = self.current_dictionary
        matched_words = []
    
        for candidate in current_words:
            if len(candidate) != word_length:
                continue
            if re.match(regex_pattern, candidate):
                matched_words.append(candidate)
    
        self.current_dictionary = matched_words

        letter_frequency = count_letter_presence(matched_words)
        sorted_letters = letter_frequency.most_common()
    
        guessed_char = '!' 
        for letter, _ in sorted_letters:
            if letter not in self.guessed_letters:
                # Skip vowels if vowel ratio is already high
                if letter in vowels and calculate_vowel_ratio(regex_pattern) > 0.55:
                    self.guessed_letters.append(letter)
                    continue
                guessed_char = letter
                break

        if guessed_char == '!':
            fallback_matches = filter_matching_words(n_word_dictionary, regex_pattern)
            fallback_freq = count_letter_presence(fallback_matches)
            fallback_sorted = fallback_freq.most_common()
    
            for letter, _ in fallback_sorted:
                if letter not in self.guessed_letters:
                    if letter in vowels and calculate_vowel_ratio(regex_pattern) > 0.55:
                        self.guessed_letters.append(letter)
                        continue
                    guessed_char = letter
                    break
        if guessed_char == '!':
            chunk_size = int(word_length / 2)
            if chunk_size >= 3:
                combined_freq = collections.Counter()
                for i in range(word_length - chunk_size + 1):
                    substring = regex_pattern[i:i+chunk_size]
                    sub_matches = filter_matching_words(n_word_dictionary, substring)
                    combined_freq += count_letter_presence(sub_matches)
    
                sorted_combined = combined_freq.most_common()
                for letter, _ in sorted_combined:
                    if letter not in self.guessed_letters:
                        guessed_char = letter
                        break

        if guessed_char == '!':
            chunk_size = int(word_length / 3)
            if chunk_size >= 3:
                combined_freq = collections.Counter()
                for i in range(word_length - chunk_size + 1):
                    substring = regex_pattern[i:i+chunk_size]
                    sub_matches = filter_matching_words(n_word_dictionary, substring)
                    combined_freq += count_letter_presence(sub_matches)
    
                sorted_combined = combined_freq.most_common()
                for letter, _ in sorted_combined:
                    if letter not in self.guessed_letters:
                        guessed_char = letter
                        break

        if guessed_char == '!':
            for letter, _ in self.full_dictionary_common_letter_sorted:
                if letter not in self.guessed_letters:
                    if letter in vowels and calculate_vowel_ratio(regex_pattern) > 0.55:
                        self.guessed_letters.append(letter)
                        continue
                    guessed_char = letter
                    break
    
        return guessed_char


    def guess(self, word):

        word = word[::2]    
        input_seq = self.encode_word(word)
        preds = self.model.predict(input_seq, verbose=0)[0]  # shape: (26,)
        sorted_indices = np.argsort(preds)[::-1]
    
        for idx in sorted_indices:
            ch = self.alphabet[idx]
            if ch not in self.guessed_letters and preds[idx] >= self.model_threshold:
                return ch
    
        return self.fallback_guess(word)

            
        

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            self.tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, self.tries_remains, word))
            while self.tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                self.tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [54]:
api = HangmanAPI(access_token="f3f3561b3c19e401f0fa7c41e484b9", timeout=2000)


C:\Users\atulp\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [53]:
n = 50
start = 0
end = 0

for _ in range(n):
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() 
    if (_ == 0):
        start = total_practice_successes
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs

    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
    
    end = total_practice_successes
    print(f'Accuracy obtained in the run so far is {(end - start) /(_+1)}')
        


Successfully start a new game! Game ID: bcfb924e0611. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ i _ _ _ i _ e '}
Guessing letter: t
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ t i _ _ t i _ e '}
Guessing letter: u
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 6, 'word': 'u _ _ u t i _ _ t i _ e '}
Guessing letter: a
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 6, 'word': 'u _ _ u t i _ a t i _ e '}
Guessing letter: o
Sever response: {'game_id': 'bcfb924e0611', 'status': 'ongoing', 'tries_remains': 5, 'word': 'u _ _ u t i _ a t i _ e '}
Guessing letter: v
Sever response: {'game_id': 'b

KeyboardInterrupt: 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)